In [1]:
%matplotlib qt
import numpy as np
import os.path as op
from pandas import read_csv
import mne
from mne.io import read_raw_fif
from mne.datasets import visual_92_categories
from neurora.nps_cal import nps
from neurora.rdm_cal import eegRDM
from neurora.rdm_corr import rdm_correlation_spearman
from neurora.corr_cal_by_rdm import rdms_corr
from neurora.rsa_plot import plot_rdm, plot_corrs_by_time, plot_nps_hotmap, plot_corrs_hotmap
import pandas as pd

from neurora.rsa_plot import plot_tbytsim_withstats

In [17]:
list_epochs_all = list()


# DD
# 去掉 20数据不全 24没数据 25无法ICA
sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4016_y","pre4017_y", "pre4018_y","pre4019_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y",
    "pre4027_y","pre4028_y","pre4029_y","pre4030_y","pre4031_y","pre4032_y",
    "pre4033_y","pre4034_y",

]
sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4016_y","pre4017_y", "pre4018_y","pre4019_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y",
    "pre4027_y","pre4028_y","pre4029_y","pre4030_y"
]
# DD组
sub_ids = [
            'pre4010_y','pre4012_y','pre4013_y','pre4015_y','pre4017_y','pre4018_y','pre4019_y','pre4021_y','pre4022_y','pre4023_y','pre4026_y','pre4027_y','pre4030_y',
          ]
# 非DD组
sub_ids = [
           'pre4008_y','pre4009_y','pre4011_y','pre4014_y','pre4028_y','pre4029_y'
         ]
# 正常组
sub_ids = [
           'pre4009_y','pre4014_y','pre4028_y','pre4029_y'
         ]
# # 正常儿童
# sub_ids = [
#
#      "pre4003_y","pre4007_y","pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y"
#
# ]

file_path = 'E:\实验\阅读困难\数据处理\pre4\data\\6epoch_clean_RSA\\'
channels_nieye = ['M1','T7','P7','TP7','M2','T8','P8','TP8','CP5','CP6','C5','C6']
channels_nieye = ['T7','P7','TP7','M2','T8','P8','TP8','CP5','CP6','C5','C6']
# 左右脑 偏侧化.
channels_left = ['Fp1',  'AF3',  'F7', 'F5', 'F3', 'F1',  'FT7', 'FC5', 'FC3', 'FC1',  'T7', 'C5', 'C3', 'C1',   'TP7', 'CP5', 'CP3', 'CP1',  'P7', 'P5', 'P3', 'P1',   'PO7', 'PO5', 'PO3',  'O1', ]

channels_right = ['Fp2','AF4',  'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT8','C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P2', 'P4', 'P6', 'P8','PO4', 'PO6', 'PO8','O2',]

# 前额叶
channels_all = ['Fp1', 'Fpz', 'Fp2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POz', 'PO4', 'PO6', 'PO8', 'O1', 'Oz', 'O2', ]

channels_all = ['Fp1', 'Fpz', 'Fp2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POz', 'PO4', 'PO6', 'PO8', 'O1', 'Oz', 'O2', ]

channels_all = channels_left
num_channel = len(channels_all)

for sub_id in sub_ids:
    data_path = file_path + sub_id + '_RSA-epo.fif'

    epochs_all = mne.read_epochs(fname=data_path)
    # 如果只想看某一部分的脑区,配合更改ndarray的维度
    epochs_all = epochs_all.pick(picks=channels_all)

    list_epochs_all.append(epochs_all)

'''
思路，先求ERP，然后再给ndarray，然后再计算RDM
'''

Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4009_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1007 matching events found
No baseline correction applied
0 projection items activated
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4014_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1332 matching events found
No baseline correction applied
0 projection items activated
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4028_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1236 matching events found
No baseline correction applied
0 projection items activated
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4029_y_RSA-epo.fif ...
    Found t

'\n思路，先求ERP，然后再给ndarray，然后再计算RDM\n'

In [18]:

'''
生成每种条件对应的evoked
而且是按照次序
'''
num_channel = len(channels_all)
conds = ['101', '102', '103', '104', '111', '112', '113', '114', '121', '122', '123', '124', '131', '132', '133', '134', ]

eegdata = np.zeros([len(sub_ids),16, num_channel, 1001], dtype=np.float32)
#初始值
subindex = 0
for m in range(len(sub_ids)):
    #单个被试的数据
    subdata = np.zeros([16, num_channel, 1001], dtype=np.float32)
    for i in range(len(conds)):
        epochs = list_epochs_all[m][conds[i]]
        evoked = epochs.average()
        data = evoked.data
        subdata[i] = data
    eegdata[m] = subdata

In [19]:
# shape of megdata: [n_subs, n_cons, n_chls, n_ts] -> [n_cons, n_subs, n_chls, n_ts]
eegdata = np.transpose(eegdata, (1, 0, 2, 3))

# shape of megdata: [n_cons, n_subs, n_chls, n_ts] -> [n_cons, n_subs, n_trials, n_chls, n_ts]
# here data is averaged, so set n_trials = 1
eegdata = np.reshape(eegdata, [16, len(sub_ids), 1, num_channel, 1001])


In [20]:
# Calculate the RDM based on the data during 190ms-210ms
#sub_opt=1 indicate不止一个被试。
#The shape of EEGdata must be [n_cons, n_subs, n_trials, n_chls, n_ts].
# Sometimes, the numbers of trials under different conditions are not same. In NeuroRA, we recommend users to average
#     the trials under a same condition firstly in this situation. Thus, the shape of input (EEG_data) should be
#     [n_cons, n_subs, 1, n_chls, n_ts].


rdm = eegRDM(eegdata[:, :, :, :, 340:350])
# Plot this RDM
plot_rdm(rdm[0], percentile=True)


Computing RDMs

RDMs computing finished!


0

In [21]:
# Calculate the RDMs by a 10ms time-window
# (raw sampling requency is 1000Hz, so here time_win=10ms/(1s/1000Hz)/1000=10)
# sub_opt = 1

# 默认窗口是50，那么就是 （1000-50）/5 = 190
#

rdms = eegRDM(eegdata, time_opt=1, time_win=50, time_step=5, sub_opt=1)


Computing RDMs
Calculating: [====================================================================================================] 100.00%
RDMs computing finished!


In [22]:
# Plot the RDM of -100ms, 0ms, 50ms, 100ms, 150ms, 200ms
times = [0, 10, ]
for t in times:
    plot_rdm(rdms[1,t], percentile=True)

## 辅音_理论模型

In [23]:

'''
辅音_理论模型
'''

RDM_model_fuyin = np.zeros([16, 16], dtype=np.float32)
# for i in range(20):
#     for j in range(20):
#         if abs(i-j)<=3:
#             RDM_model_fuyin[i,j] = 0
#         else:
#             RDM_model_fuyin[i,j] =
zeroM = np.zeros([4, 4], dtype=np.float32)
oneM = np.ones((4, 4), dtype=np.float32)

a1 = np.concatenate((zeroM, oneM, oneM, oneM), axis=1)
a2 = np.concatenate((oneM, zeroM, oneM, oneM), axis=1)
a3 = np.concatenate((oneM, oneM, zeroM, oneM), axis=1)
a4 = np.concatenate((oneM, oneM, oneM, zeroM), axis=1)

RDM_model_fuyin = np.concatenate((a1, a2, a3, a4), axis=0)
plot_rdm(RDM_model_fuyin, percentile=True)
# Calculate the representational similarity between 200ms and all the time points
# （sub,times,2）


0

In [24]:
corrs_EEGvsFuyin = rdms_corr(RDM_model_fuyin, rdms)
%matplotlib qt
# Plot the corrs1
# corrs1 = np.reshape(corrs1, [1, 191, 2])
# plot_corrs_by_time(corrs1, time_unit=[-0.2, 0.005])
from neurora.rsa_plot import plot_tbytsim_withstats
from neurora.stuff import clusterbased_permutation_1d_1samp_1sided

#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
# start_time =
ps_EEGvsFuyin = plot_tbytsim_withstats(corrs_EEGvsFuyin, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,
                                       clusterp=0.05, stats_time=[0, 0.8], color='b', xlim=(-0.2, 1), fontsize=9)

# plot_tbytsim_withstats 内嵌了  clusterbased_permutation_1d_1samp_1sided,统计部分由这个函数来完成的.
# similarities = corrs_EEGvsFuyin[:, :, 0]
# clusterbased_permutation_1d_1samp_1sided(similarities)


Computing similarities

Computing finished!

Significant time-windows:


### 保存结果


In [25]:
'''
保存corrs_EEGvsFuyin

1 需要算出极大值和对应的时间点-算峰值的方法 ---为了和行为做相关.

2 算显著时间点均值

'''

# 问题:这里应该保存 -200 - 800ms的 还是保存 显著的时间窗口内的呢?
# 应该保存窗口内的,不然没有做的意义了.
# idea 可以用 ps来进行窗口的限制 截取对应的时间窗
npy_path = './脑电结果/'
type1 = 'rdms'
type2 = 'corrs'
type3 = 'ps'

conditon1 = 'Fuyin_非DD'

x = npy_path+type1+'_'+conditon1+'.npy'

np.save('./脑电结果/rdms_EEGvsFuyin_非DD.npy', rdms)
np.save('./脑电结果/corrs_EEGvsFuyin_非DD.npy', corrs_EEGvsFuyin)
np.save('./脑电结果/ps_EEGvsFuyin_非DD.npy', ps_EEGvsFuyin)


In [36]:
# 假设你有一些变量
var1 = 1
var2 = 2
rdms_var = 3
other_var = 4

# 获取所有全局变量的名字
global_vars = globals()

# 找到包含'rdms'的变量名
rdms_vars = [name for name, value in global_vars.items() if 'rdms_EEG' in name]

# 输出结果
print(rdms_vars)


[]


## 音调_理论模型

In [26]:
'''
音调 模型
'''

RDM_model_yindiao = np.zeros([16,16], dtype=np.float32)


oneM =  np.ones((4, 4), dtype=np.float32)
for i in range(4):
    oneM[i][i] = 0
zeroM = oneM

a1=np.concatenate((zeroM,oneM,oneM,oneM,),axis = 1)
a2=np.concatenate((oneM,zeroM,oneM,oneM,),axis = 1)
a3=np.concatenate((oneM,oneM,zeroM,oneM,),axis = 1)
a4=np.concatenate((oneM,oneM,oneM,zeroM,),axis = 1)

RDM_model_yindiao = np.concatenate((a1,a2,a3,a4),axis=0)

plot_rdm(RDM_model_yindiao, percentile=True)

0

In [27]:
# Calculate the representational similarity between 200ms and all the time points
corrs_EEGvsYindiao = rdms_corr(RDM_model_yindiao, rdms)

from neurora.rsa_plot import plot_tbytsim_withstats

#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
ps_EEGvsYindiao = plot_tbytsim_withstats(corrs_EEGvsYindiao, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05,stats_time=[0,0.8], xlim=(-0.2,1), fontsize=9)



Computing similarities

Computing finished!

Permutation test
Calculating: [====================================================================================================] 100.00%
Cluster-based permutation test finished!


Significant time-windows:
325ms to 345ms
375ms to 470ms


## 行为模型


In [28]:

'''
基于行为数据的model RDM
'''

from scipy.io import loadmat

# 指定要读取的 .mat 文件路径
file_path_tone = "E:\实验\阅读困难\数据处理\pre4\RSA\models矩阵\\toneRDM.mat"  # 将 "path/to/your/file.mat" 替换为实际的文件路径
# 使用 loadmat 函数读取 .mat 文件
RDM_model_yindiao_xw = loadmat(file_path_tone)['final_rdm']

# 指定要读取的 .mat 文件路径
file_path_consonat = "E:\实验\阅读困难\数据处理\pre4\RSA\models矩阵\\consonantRDM.mat"  # 将 "path/to/your/file.mat" 替换为实际的文件路径
# 使用 loadmat 函数读取 .mat 文件
RDM_model_fuyin_xw = loadmat(file_path_consonat)['new_rdm']

In [34]:
# Calculate the representational similarity between 200ms and all the time points
corrs_EEGvsFuyin_behavior = rdms_corr(RDM_model_fuyin_xw, rdms)
#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
ps_corrs_EEGvsFuyin_behavior = plot_tbytsim_withstats(corrs_EEGvsFuyin_behavior, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05,stats_time=[0,0.8], xlim=[-0.2,1], fontsize=9, color='b')


Computing similarities

Computing finished!

Significant time-windows:


In [35]:
from neurora.rsa_plot import plot_tbytsim_withstats
corrs_EEGvsYindiao_behavior = rdms_corr(RDM_model_yindiao_xw, rdms)
#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
ps_corrs_EEGvsYindiao_behavior = plot_tbytsim_withstats(corrs_EEGvsYindiao_behavior, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05, color='r',stats_time=[0,0.8], xlim=[-0.2,1], fontsize=9)




Computing similarities

Computing finished!

Permutation test
Calculating: [====================================================================================================] 100.00%
Cluster-based permutation test finished!


Significant time-windows:
390ms to 434ms


### 结果保存

In [17]:
np.save('./脑电结果/corrs_EEGvsFuyin_behavior_非DD.npy', corrs_EEGvsFuyin_behavior)
np.save('./脑电结果/ps_corrs_EEGvsFuyin_behavior_非DD.npy', ps_corrs_EEGvsFuyin_behavior)